In [0]:
data = [(1,"Shirish",10000),(2,"Meguer",30000),(3,"Satish",20000)]
myschema = ["id","name","salary"]

In [0]:
df = spark.createDataFrame(data,myschema)
df.display()

id,name,salary
1,Shirish,10000
2,Meguer,30000
3,Satish,20000


In [0]:
import timeit
start = timeit.default_timer()
df.collect()
end=timeit.default_timer()
print("elapsed time: {}".format(end-start))

elapsed time: 0.6834274610009743


In [0]:
df.cache()

Out[6]: DataFrame[id: bigint, name: string, salary: bigint]

In [0]:
import timeit
start = timeit.default_timer()
df.collect()
end=timeit.default_timer()
print("elapsed time: {}".format(end-start))

elapsed time: 0.38098125199940114


In [0]:
df.unpersist()

Out[8]: DataFrame[id: bigint, name: string, salary: bigint]

In [0]:
import timeit
start = timeit.default_timer()
df.collect()
end=timeit.default_timer()
print("elapsed time: {}".format(end-start))

elapsed time: 0.6291176380000252


In [0]:
data1 = [(1,"Shirish",10000),(2,"Meguer",30000),(3,"Satish",20000),(4,"Jasmine",10000),(5,"Anusha",30000),(6,"Venkat",20000),(7,"Devashish",10000),(8,"Srijana",30000),(9,"Prakhyat",20000),(10,"Ranju",1500000)]
myschema1 = ["id","name","salary"]

In [0]:
df2 = spark.createDataFrame(data1,myschema1)
df2.display()

id,name,salary
1,Shirish,10000
2,Meguer,30000
3,Satish,20000
4,Jasmine,10000
5,Anusha,30000
6,Venkat,20000
7,Devashish,10000
8,Srijana,30000
9,Prakhyat,20000
10,Ranju,1500000


Return all employees having second highest salary

In [0]:
df2.createOrReplaceTempView("employees")

query = """
SELECT ID, Name, Salary
FROM (
    SELECT ID, Name, Salary, DENSE_RANK() OVER (ORDER BY Salary DESC) AS rank
    FROM employees
) ranked
WHERE rank = 2
"""

second_highest_salary_sql = spark.sql(query)

second_highest_salary_sql.show()

+---+-------+------+
| ID|   Name|Salary|
+---+-------+------+
|  2| Meguer| 30000|
|  5| Anusha| 30000|
|  8|Srijana| 30000|
+---+-------+------+



In [0]:
from pyspark.sql.functions import col, dense_rank
from pyspark.sql.window import Window

window_spec = Window.orderBy(col("salary").desc())

df_with_rank = df2.withColumn("rank", dense_rank().over(window_spec))

second_highest_salary = df_with_rank.filter(col("rank") == 2)

second_highest_salary.show()

+---+-------+------+----+
| id|   name|salary|rank|
+---+-------+------+----+
|  2| Meguer| 30000|   2|
|  5| Anusha| 30000|   2|
|  8|Srijana| 30000|   2|
+---+-------+------+----+

